# Install the library
You must follow the instructions from the MMSegmentation webpage [Installation Guide](https://github.com/open-mmlab/mmsegmentation/blob/main/docs/en/get_started.md#installation).

Be sure to have the mmsegmentation in the same directory as the project, since we use the Config files from the library.

# Imports

In [1]:
import json
import pandas as pd
import os
import mmcv
import mmengine
import matplotlib.pyplot as plt
import numpy as np
from mmengine import Config
from PIL import Image
import matplotlib.patches as mpatches
from mmseg.registry import DATASETS
from mmseg.datasets import BaseSegDataset

# Create text files with the images to be processed in the training

In [ ]:
# Open the json file and load the data
with open('../_data/fashionpedia/ann_dir/instances_attributes_train2020.json') as f:
    data_train = json.load(f)
    images_train = pd.DataFrame(data_train['images'])
    annotations_train = pd.DataFrame(data_train['annotations'])
    categories_train = pd.DataFrame(data_train['categories'])

with open('../_data/fashionpedia/ann_dir/instances_attributes_val2020.json') as f:
    data_val = json.load(f)
    images_val = pd.DataFrame(data_val['images'])
    annotations_val = pd.DataFrame(data_val['annotations'])
    categories_val = pd.DataFrame(data_val['categories'])

In [ ]:
# Save the name file without the extension
images_train['file_name'] = images_train['file_name'].apply(lambda x: os.path.splitext(x)[0])
images_val['file_name'] = images_val['file_name'].apply(lambda x: os.path.splitext(x)[0])

# Check if the segmentation file exists
for index, row in images_train.iterrows():
    filename = row['file_name']
    file_path = '../_data/fashionpedia/ann_dir/train/' + filename.split('.')[0] + '_seg.png'
    if not os.path.exists(file_path):
        images_train.drop(index, inplace=True)

for index, row in images_val.iterrows():
    filename = row['file_name']
    file_path = '../_data/fashionpedia/ann_dir/val/' + filename.split('.')[0] + '_seg.png'
    if not os.path.exists(file_path):
        images_val.drop(index, inplace=True)

In [ ]:
# Save the files with the file_names
filenames_train = images_train['file_name']
filenames_train.to_csv('../_data/fashionpedia/train_files.txt', sep='\t', index=False, header=False)

filenames_val = images_val['file_name']
filenames_val.to_csv('../_data/fashionpedia/val_files.txt', sep='\t', index=False, header=False)

# Train

In [2]:
def generate_palette(num_classes, seed=42):
    """
    Generate a palette for MMSegmentation given the number of classes.

    Args:
        num_classes (int): Number of classes.

    Returns:
        list: Palette list of tuples of length `num_classes` in BGR format.
    """
    np.random.seed(seed)
    palette = []

    # Generate BGR colors
    for i in range(num_classes):
        if i == 0:
            # The first color is reserved for background
            palette.append((0, 0, 0))  # Background color (Black)
            continue
        else:
            b = np.random.randint(0, 256)
            g = np.random.randint(0, 256)
            r = np.random.randint(0, 256)
            palette.append((b, g, r))  # BGR format

    return palette

## Variable setup

In [3]:
# 0 = normal data, 1 = reduced data, 2 = data augmentation
normal_or_reduced = 1

data_root = ['../_data/fashionpedia', '../_data/fashionpedia-reduction', '../fashionpedia-daug'][normal_or_reduced]
img_dir = 'img_dir'
ann_dir = 'ann_dir'

classes = [('background', 'shirt, blouse', 'top, t-shirt, sweatshirt', 'sweater', 'cardigan', 'jacket', 'vest', 'pants', 'shorts', 'skirt', 'coat', 'dress', 'jumpsuit', 'cape', 'glasses', 'hat', 'headband, head covering, hair accessory', 'tie', 'glove', 'watch', 'belt', 'leg warmer', 'tights, stockings', 'sock', 'shoe', 'bag, wallet', 'scarf', 'umbrella', 'hood', 'collar', 'lapel', 'epaulette', 'sleeve', 'pocket', 'neckline', 'buckle', 'zipper', 'applique', 'bead', 'bow', 'flower', 'fringe', 'ribbon', 'rivet', 'ruffle', 'sequin', 'tassel'),
           ('background', 'vest', 'headband, head covering, hair accessory', 'tie', 'glove',
       'watch', 'leg warmer', 'sock', 'scarf', 'hood', 'epaulette',
       'pocket', 'buckle', 'zipper', 'applique', 'bead', 'bow', 'flower',
       'fringe', 'ribbon', 'rivet', 'ruffle', 'sequin', 'tassel'),
       ('background', 'shirt, blouse', 'top, t-shirt, sweatshirt', 'sweater', 'cardigan', 'jacket', 'vest', 'pants', 'shorts', 'skirt', 'coat', 'dress', 'jumpsuit', 'cape', 'glasses', 'hat', 'headband, head covering, hair accessory', 'tie', 'glove', 'watch', 'belt', 'leg warmer', 'tights, stockings', 'sock', 'shoe', 'bag, wallet', 'scarf', 'umbrella', 'hood', 'collar', 'lapel', 'epaulette', 'sleeve', 'pocket', 'neckline', 'buckle', 'zipper', 'applique', 'bead', 'bow', 'flower', 'fringe', 'ribbon', 'rivet', 'ruffle', 'sequin', 'tassel')][normal_or_reduced]
palette = generate_palette(len(classes))

## Add the new dataset instance

In [5]:
@DATASETS.register_module()
class FashionpediaDataset(BaseSegDataset):
  METAINFO = dict(classes = classes, palette = palette)
  def __init__(self, **kwargs):
    super().__init__(img_suffix='.jpg', seg_map_suffix='_seg.png', **kwargs)

## Setup the training variables

In [6]:
chosen_architecture = [0, 1, 2, 3, 4][0]
architecture = ['ocrnet/ocrnet_hr18s_4xb4-80k_ade20k-512x512.py', 'ocrnet/ocrnet_r101-d8_4xb2-40k_cityscapes-512x1024.py', 'dmnet/dmnet_r50-d8_4xb2-40k_cityscapes-512x1024.py', 'point_rend/pointrend_r50_4xb2-80k_cityscapes-512x1024.py', 'point_rend/pointrend_r50_4xb4-160k_ade20k-512x512.py'][chosen_architecture]
weights = ['ocrnet_hr18s_512x512_80k_ade20k_20200615_055600-e80b62af.pth', 'ocrnet_r101-d8_512x1024_40k_b8_cityscapes_20200717_110721-02ac0f13.pth', 'dmnet_r50-d8_512x1024_40k_cityscapes_20201215_042326-615373cf.pth', 'pointrend_r50_512x1024_80k_cityscapes_20200711_015821-bb1ff523.pth', 'pointrend_r50_512x512_160k_ade20k_20200807_232644-ac3febf2.pth'][chosen_architecture]
batch_size = [2, 4, 8, 16][0]
image_resolution = [192, 384][0]
train_annotation_file = ['train_files.txt', 'reduced_train_files.txt'][normal_or_reduced]
val_annotation_file = ['val_files.txt', 'reduced_val_files.txt'][normal_or_reduced]

# Load and setup the config for the model

In [13]:
cfg = Config.fromfile('../mmsegmentation/configs/' + architecture)
print(f'Config:\n{cfg.pretty_text}')

Config:
crop_size = (
    512,
    512,
)
data_preprocessor = dict(
    bgr_to_rgb=True,
    mean=[
        123.675,
        116.28,
        103.53,
    ],
    pad_val=0,
    seg_pad_val=255,
    size=(
        512,
        512,
    ),
    std=[
        58.395,
        57.12,
        57.375,
    ],
    type='SegDataPreProcessor')
data_root = 'data/ade/ADEChallengeData2016'
dataset_type = 'ADE20KDataset'
default_hooks = dict(
    checkpoint=dict(by_epoch=False, interval=16000, type='CheckpointHook'),
    logger=dict(interval=50, log_metric_by_epoch=False, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    timer=dict(type='IterTimerHook'),
    visualization=dict(type='SegVisualizationHook'))
default_scope = 'mmseg'
env_cfg = dict(
    cudnn_benchmark=True,
    dist_cfg=dict(backend='nccl'),
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0))
img_ratios = [
    0.5,
    0.75,
    1.0,
    1.25,
    1.5

In [14]:
# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.crop_size = (image_resolution, image_resolution)
cfg.model.data_preprocessor.size = cfg.crop_size
cfg.model.backbone.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head
if architecture.split('/')[0] == 'ocrnet' or architecture.split('/')[0] == 'point_rend':
    cfg.model.decode_head[0].num_classes = len(classes)
    cfg.model.decode_head[1].num_classes = len(classes)
else:
    cfg.model.decode_head.num_classes = len(classes)
    cfg.model.auxiliary_head.num_classes = len(classes)

# Modify dataset type and path
cfg.dataset_type = 'FashionpediaDataset'
cfg.data_root = data_root

cfg.train_dataloader.batch_size = batch_size

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='RandomResize', scale=(image_resolution, image_resolution), ratio_range=(0.5, 2.0), keep_ratio=True),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackSegInputs')
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(image_resolution, image_resolution), keep_ratio=True),
    # add loading annotation after ``Resize`` because ground truth
    # does not need to do resize data transform
    dict(type='LoadAnnotations'),
    dict(type='PackSegInputs')
]


cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = dict(img_path=img_dir+'/train', seg_map_path=ann_dir+'/train')
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
cfg.train_dataloader.dataset.ann_file = train_annotation_file
cfg.train_dataloader.persistent_workers = False
cfg.train_dataloader.num_workers = 0
cfg.train_dataloader.sampler.type= "DefaultSampler"

cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = dict(img_path=img_dir+'/val', seg_map_path=ann_dir+'/val')
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.val_dataloader.dataset.ann_file = val_annotation_file
cfg.val_dataloader.persistent_workers = False
cfg.val_dataloader.num_workers = 0
cfg.val_evaluator.iou_metrics = ['mDice']

cfg.test_dataloader = cfg.val_dataloader


# Load the pretrained weights
cfg.load_from = weights

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial/' + architecture.split('/')[0] + '_' + str(batch_size) + 'bs_' + str(image_resolution) + 'ir'

cfg.train_cfg = dict(
    by_epoch=True,
    max_epochs=2,
    val_interval=1)

cfg.default_hooks.checkpoint.by_epoch = True
cfg.default_hooks.checkpoint.interval = 1
cfg.default_hooks.logger = dict(type='LoggerHook', log_metric_by_epoch=True, out_suffix='.json')
cfg.log_config = dict(
    interval=50,
    hooks=[
        dict(type='TextLoggerHook')
    ])

cfg.log_processor.by_epoch = True

cfg.param_scheduler[0].by_epoch = True

# Set seed to facilitate reproducing the result
cfg['randomness'] = dict(seed=0)

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
crop_size = (
    384,
    384,
)
data_preprocessor = dict(
    bgr_to_rgb=True,
    mean=[
        123.675,
        116.28,
        103.53,
    ],
    pad_val=0,
    seg_pad_val=255,
    size=(
        512,
        512,
    ),
    std=[
        58.395,
        57.12,
        57.375,
    ],
    type='SegDataPreProcessor')
data_root = '../_data/fashionpedia-reduction'
dataset_type = 'FashionpediaDataset'
default_hooks = dict(
    checkpoint=dict(by_epoch=True, interval=1, type='CheckpointHook'),
    logger=dict(
        log_metric_by_epoch=True, out_suffix='.json', type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    timer=dict(type='IterTimerHook'),
    visualization=dict(type='SegVisualizationHook'))
default_scope = 'mmseg'
env_cfg = dict(
    cudnn_benchmark=True,
    dist_cfg=dict(backend='nccl'),
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0))
img_ratios = [
    0.5,
    0.75,
    1.0,

In [15]:
from mmengine.runner import Runner

runner = Runner.from_cfg(cfg)

04/19 15:06:33 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.8.19 (default, Mar 20 2024, 19:55:45) [MSC v.1916 64 bit (AMD64)]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 0
    GPU 0: NVIDIA GeForce GTX 1650
    CUDA_HOME: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v.12.3
    MSVC: n/a, reason: fileno
    PyTorch: 2.0.0
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 199711
  - MSVC 193431937
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-ge

c:\Users\Laia\miniconda3\envs\openmmlab\lib\site-packages\mmengine\utils\manager.py:113: UserWarning: <class 'mmseg.visualization.local_visualizer.SegLocalVisualizer'> instance named of visualizer has been created, the method `get_instance` should not accept any other arguments
  warnings.warn(


04/19 15:06:34 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
04/19 15:06:34 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

In [16]:
runner.train()

04/19 15:06:38 - mmengine - WARNING - The prefix is not set in metric class IoUMetric.
04/19 15:06:38 - mmengine - INFO - load model from: open-mmlab://resnet50_v1c
04/19 15:06:38 - mmengine - INFO - Loads checkpoint by openmmlab backend from path: open-mmlab://resnet50_v1c
04/19 15:06:39 - mmengine - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

Loads checkpoint by local backend from path: pointrend_r50_512x512_160k_ade20k_20200807_232644-ac3febf2.pth
The model and loaded state dict do not match exactly

size mismatch for decode_head.0.conv_seg.weight: copying a param with shape torch.Size([150, 128, 1, 1]) from checkpoint, the shape in current model is torch.Size([24, 128, 1, 1]).
size mismatch for decode_head.0.conv_seg.bias: copying a param with shape torch.Size([150]) from checkpoint, the shape in current model is torch.Size([24]).
size mismatch for decode_head.1.fcs.0.conv.weight: copying a param with shap

CascadeEncoderDecoder(
  (data_preprocessor): SegDataPreProcessor()
  (backbone): ResNetV1c(
    (stem): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, e

In [ ]:
from mmseg.apis import init_model, inference_model, show_result_pyplot

# Init the model from the config and the checkpoint
checkpoint_path = './work_dirs/tutorial/' + architecture.split('/')[0] + '_' + str(batch_size) + 'bs_' + str(image_resolution) + 'ir/epoch_2.pth'
model = init_model(cfg, checkpoint_path, 'cuda:0')
model.dataset_meta = dict(classes = classes, palette = palette)
img = mmcv.imread('../_data/fashionpedia/img_dir/val/0bbaa62211cc48b57957aab152fba023.jpg')

result = inference_model(model, img)
plt.figure(figsize=(8, 6))
vis_result = show_result_pyplot(model, img, result)
plt.imshow(mmcv.bgr2rgb(vis_result))